Setting up:

In [1]:
import numpy as np
import torch

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

In [2]:
%pip install pennylane

Note: you may need to restart the kernel to use updated packages.


Define a qnode in `pennylane`:

In [3]:
import pennylane as qml

n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    # Embedding
    qml.AngleEmbedding(inputs, wires=range(n_qubits))

    # Entanglement
    for j in range(weights.shape[0]):
        for i in range(weights.shape[1]):
            qml.RY(weights[j][i], wires=i)

        for i in range(weights.shape[1] - 1):
            qml.CNOT(wires=[i, i + 1])

        qml.CNOT(wires=[weights.shape[1] - 1, 0])

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(weights.shape[1])]

In [4]:
# print(qml.draw(qnode)(inputs=np.random.rand(4), weights=np.random.randn(4, 4)))

For the QNode to be successfully converted to a layer in `torch.nn`, we need to provide the details of the shape of each trainable weight for them to be initialized. The weight_shapes dictionary maps from the argument names of the QNode to corresponding shapes:

In [5]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}

Now that `weight_shapes` is defined, it is easy to then convert the QNode:

In [6]:
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

With this done, the QNode can now be treated just like any other torch.nn layer and we can proceed using the familiar Torch workflow.

## Creating a hybrid model
Since our text is already embedded, our hybrid model will consist of:

1) two fully connected classical layers: 768 -> 128 and 128 -> 8
2) an 8-qubit QNode converted into a layer
3) a fully connected classical layer: 8 -> 1
4) a sigmoid

In [7]:
clayer_1 = torch.nn.Linear(768, 128)
clayer_2 = torch.nn.Linear(128, n_qubits)
relayer_1 = torch.nn.LeakyReLU(0.2)
clayer_3 = torch.nn.Linear(n_qubits, 1)
softmax = torch.nn.Sigmoid()
layers = [clayer_1, clayer_2, qlayer, clayer_3, softmax]
model = torch.nn.Sequential(*layers)
model

Sequential(
  (0): Linear(in_features=768, out_features=128, bias=True)
  (1): Linear(in_features=128, out_features=8, bias=True)
  (2): <Quantum Torch Layer: func=qnode>
  (3): Linear(in_features=8, out_features=1, bias=True)
  (4): Sigmoid()
)

## Training the model
We can now train our hybrid model on the classification dataset using the usual Torch approach. We’ll use the standard `SGD` optimizer and the mean absolute error loss function:

In [8]:
opt = torch.optim.SGD(model.parameters(), lr=0.2)
loss = torch.nn.BCELoss()

In [9]:
X = torch.load(open("data/X.pt", 'rb'))
y = torch.load(open("data/y.pt", 'rb'))

batch_size = 20

data = list(zip(X, y))
data_train, data_test = torch.utils.data.random_split(data, [0.8, 0.2])

data_loader = torch.utils.data.DataLoader(
    data_train, batch_size=20, shuffle=True, drop_last=False
)
test_loader = torch.utils.data.DataLoader(dataset=data_test, shuffle=False)

epochs = 20
accuracies = []

for epoch in range(epochs):

    running_loss = 0

    for xs, ys in data_loader:
        opt.zero_grad()
        loss_evaluated = loss(model(xs).squeeze(), ys.float())
        loss_evaluated.backward()

        opt.step()

        running_loss += loss_evaluated

    avg_loss = running_loss / len(y)
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))

    correct = 0
    for xt, yt in test_loader:
        # print(model(xt), yt)
        correct += (model(xt) >= 0.5) == yt
    accuracy = correct / len(test_loader)
    accuracies.append(accuracy.item())
    print(f"Validation accuracy: {accuracy.item() * 100:.2f}%")

C:\Users\user\AppData\Local\Temp\ipykernel_7012\795876464.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load(open("data/X.pt", 'rb'))
C:\Users\user\AppData\L

Average loss over epoch 1: 0.0289
Validation accuracy: 45.24%
Average loss over epoch 2: 0.0280
Validation accuracy: 45.24%
Average loss over epoch 3: 0.0269
Validation accuracy: 45.24%
Average loss over epoch 4: 0.0263
Validation accuracy: 45.24%
Average loss over epoch 5: 0.0252
Validation accuracy: 45.24%
Average loss over epoch 6: 0.0245
Validation accuracy: 45.24%
Average loss over epoch 7: 0.0240
Validation accuracy: 52.38%
Average loss over epoch 8: 0.0236
Validation accuracy: 71.43%
Average loss over epoch 9: 0.0227
Validation accuracy: 66.67%
Average loss over epoch 10: 0.0220
Validation accuracy: 73.81%
Average loss over epoch 11: 0.0208
Validation accuracy: 73.81%
Average loss over epoch 12: 0.0212
Validation accuracy: 85.71%
Average loss over epoch 13: 0.0197
Validation accuracy: 100.00%
Average loss over epoch 14: 0.0201
Validation accuracy: 45.24%
Average loss over epoch 15: 0.0221
Validation accuracy: 80.95%
Average loss over epoch 16: 0.0184
Validation accuracy: 85.71%


In [10]:
print(accuracies[:20])

test_loader = torch.utils.data.DataLoader(dataset=data_test, shuffle=False)
correct = 0
for xt, yt in test_loader:
    correct += (model(xt) >= 0.5) == yt
accuracy = correct / len(test_loader)
print(f"Accuracy: {accuracy.item() * 100:.2f}%")

[0.4523809552192688, 0.4523809552192688, 0.4523809552192688, 0.4523809552192688, 0.4523809552192688, 0.4523809552192688, 0.523809552192688, 0.7142857313156128, 0.6666666865348816, 0.738095223903656, 0.738095223903656, 0.8571428656578064, 1.0, 0.4523809552192688, 0.8095238208770752, 0.8571428656578064, 0.9047619104385376, 0.9047619104385376, 0.6428571343421936, 0.6428571343421936]
Accuracy: 64.29%


Save the model:

In [11]:
# torch.save(model.state_dict(), open("new_good_quantum.pt", 'wb'))

Load the best model and run it on the new test set:

In [12]:

model.load_state_dict(torch.load("pretrained_quantum\good_quantum_100.pt", weights_only=True))

X = torch.load(open("data/X_test.pt", 'rb'))
y = torch.load(open("data/y_test.pt", 'rb'))

data = list(zip(X, y))
test50_loader = torch.utils.data.DataLoader(
    data, shuffle=False, drop_last=False
)

correct = 0
for xt, yt in test50_loader:
    correct += (model(xt) >= 0.5) == yt
accuracy = correct / len(test50_loader)
print(f"Accuracy: {accuracy.item() * 100:.2f}%")

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\user\AppData\Local\Temp\ipykernel_7012\2178871295.py:1: SyntaxWarning: invalid escape sequence '\g'
  model.load_state_dict(torch.load("pretrained_quantum\good_quantum_100.pt", weights_only=True))
C:\Users\user\AppData\Local\Temp\ipykernel_7012\2178871295.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `tor

Accuracy: 80.39%
